# Setting Up All Artifacts details

In [ ]:
## Give appropriate permission to the directory "FOLDER_WITH_ARTIFACTS" you are working with
import os
os.environ['QNN_SDK_ROOT']="/local/mnt/workspace/qairt/2.40.0.251030"#set up your snpe path here.
os.environ['ANDROID_NDK_ROOT']="/usr/android-ndk-r26c"
os.environ['QNN_TARGET_ARCH']="aarch64-android"

####  Any issue with circular import error while importing super_gradients. To overcome this make the below changes ####
In your python enviroment, update super gradients package (3.1.2) package as shown below

***Code change 1#***
In /usr/venv/lib/python3.10/site-packages/super_gradients/training/\_\_init\_\_.py

**Existing Code**: import super_gradients.training.utils.distributed_training_utils as distributed_training_utils 

**Modify**: from .utils import distributed_training_utils

In [ ]:
## Note- Use python3.8 or above for generating onnx
# !pip install super-gradients==3.1.2 ### Uncomment and run it if you haven't installed it before
import torch
from super_gradients.training import models
from super_gradients.common.object_names import Models
import cv2
import numpy as np
import os

## Getting the ONNX Model

In [ ]:
os.makedirs('models', exist_ok=True)

### YOLO_NAS_S Model has issues while download. To overcome, follow below steps:
In your python enviroment, update super gradients package (3.1.2) with the modified YOLO_NAS_S download link.

***Code Change 1#***:
In /usr/venv/lib/python3.10/site-packages/super_gradients/training/pretrained_models.py line 47

**Existing Code**: "yolo_nas_s_coco": "https://sghub.deci.ai/models/yolo_nas_s_coco.pth",

**Modify**: "yolo_nas_s_coco": "https://sg-hub-nv.s3.amazonaws.com/models/yolo_nas_s_coco.pth",

***Code Change 2#***:
In /usr/venv/lib/python3.10/site-packages/super_gradient/training/utils/checkpoint_utils.py line 316, Function Name:load_pretrained_weightsline

**Existing code**: unique_filename = url.split("https://sghub.deci.ai/models/")[1].replace("/", "_").replace(" ", "_")

**Modify**: unique_filename = url.split("https://sg-hub-nv.s3.amazonaws.com/models/")[1].replace("/", "_").replace(" ", "_")

In [ ]:
model = models.get(Models.YOLO_NAS_S, pretrained_weights="coco")
# Prpare model for conversion
# Input size is in format of [Batch x Channels x Width x Height] where 640 is the standard COCO dataset dimensions
model.eval()
model.prep_model_for_conversion(input_size=[1, 3, 320, 320])
# Create dummy_input
dummy_input = torch.randn([1, 3, 320, 320], device="cpu")
# Convert model to onnx
torch.onnx.export(model, dummy_input, "models/yolo_nas_s.onnx", opset_version=11)

#### Getting the FP32 Model

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
export PATH=${ANDROID_NDK_ROOT}:${PATH}
qnn-onnx-converter --input_network models/yolo_nas_s.onnx --output_path output_cpu/yolo_nas_s.cpp --out_node 885 --out_node 877
qnn-model-lib-generator -c output_cpu/yolo_nas_s.cpp -b output_cpu/yolo_nas_s.bin --output_dir output_cpu/ -t aarch64-android
mv output_cpu/aarch64-android/libyolo_nas_s.so  output_cpu/aarch64-android/libyolo_nas_w8a8.so 

### Getting The dataset
Please, fill coco dataset link in below code block. You might need 10-15 images for quantization.

In [ ]:
!wget http://images.cocodataset.org/zips/val2017.zip -q --show-progress
!unzip val2017.zip
!mkdir "raw"

In [ ]:
files = os.listdir('val2017') #val2017 is the datatset folder path. Keeping only 15 images.
for file in files[15:]:
    os.remove("val2017/"+file)

In [ ]:
%%bash
rm -rf val2017.zip

#### Getting the Quatized Model for DSP

In [ ]:
import cv2
import numpy as np
import os

def preprocess(original_image):
    resized_image = cv2.resize(original_image, (320, 320))
    resized_image = resized_image/255
    return resized_image

##Please download Coco2014 dataset and give the path here
dataset_path = "val2017/"

filenames=[]
for path in os.listdir(dataset_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dataset_path, path)):
        filenames.append(os.path.join(dataset_path, path))
        
for filename in filenames:
    original_image = cv2.imread(filename)
    img = preprocess(original_image)
    img = img.astype(np.float32)
    img.tofile("raw/"+filename.split("/")[-1].split(".")[0]+".raw")


In [ ]:
%%bash
find raw -name *.raw > input.txt
cat input.txt

In [ ]:
%%bash
source $SNPE_ROOT/bin/envsetup.sh
export PATH=${ANDROID_NDK_ROOT}:${PATH}
qnn-onnx-converter --input_network models/yolo_nas_s.onnx --output_path output_dsp/yolo_nas_s_quantized.cpp --out_node 885 --out_node 877 --input_list input.txt 
qnn-model-lib-generator -c output_dsp/yolo_nas_s_quantized.cpp -b output_dsp/yolo_nas_s_quantized.bin --output_dir output_dsp/ -t aarch64-android
mv output_dsp/aarch64-android/libyolo_nas_s_quantized.so  output_dsp/aarch64-android/libyolo_nas_w8a8_dsp.so 